# Process Multi-Season Data - All Positions

This notebook processes raw FBref data from all positions for the Siamese neural network.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "multiseason_data"

print(f"Raw data directory: {RAW_DIR}")
print(f"Files: {list(RAW_DIR.glob('merged_*.csv'))}")

## 1. Load All Stat Tables

In [ ]:
# Load all merged stat tables
standard = pd.read_csv(RAW_DIR / "merged_standard.csv")
passing = pd.read_csv(RAW_DIR / "merged_passing.csv")
defense = pd.read_csv(RAW_DIR / "merged_defense.csv")
possession = pd.read_csv(RAW_DIR / "merged_possession.csv")
shooting = pd.read_csv(RAW_DIR / "merged_shooting.csv")
misc = pd.read_csv(RAW_DIR / "merged_misc.csv")

print(f"Standard: {standard.shape}")
print(f"Passing: {passing.shape}")
print(f"Defense: {defense.shape}")
print(f"Possession: {possession.shape}")
print(f"Shooting: {shooting.shape}")
print(f"Misc: {misc.shape}")

In [ ]:
# Check position distribution
print("Position Distribution:")
print(standard['pos'].value_counts())

## 2. Check for Duplicates in Raw Data

In [ ]:
# Check for duplicates BEFORE merging
print("=" * 60)
print("DUPLICATE CHECK - Raw Data")
print("=" * 60)

# Check exact duplicates
exact_dups = standard.duplicated().sum()
print(f"\nExact duplicate rows: {exact_dups}")

# Check duplicates by key columns
key_cols = ['player', 'team', 'season', 'league']
key_dups = standard.duplicated(subset=key_cols).sum()
print(f"Duplicate by [player, team, season, league]: {key_dups}")

# Check same player appearing multiple times in same season (different teams = transfer)
player_season_dups = standard.groupby(['player', 'season']).size()
multi_team = player_season_dups[player_season_dups > 1]
print(f"\nPlayers at multiple teams in same season: {len(multi_team)}")
if len(multi_team) > 0:
    print("Examples:")
    for (player, season), count in multi_team.head(5).items():
        teams = standard[(standard['player'] == player) & (standard['season'] == season)]['team'].tolist()
        print(f"  {player} ({season}): {teams}")

In [ ]:
# Remove exact duplicates from all tables
print("\nRemoving exact duplicates...")
before = len(standard)
standard = standard.drop_duplicates()
print(f"Standard: {before} -> {len(standard)}")

before = len(passing)
passing = passing.drop_duplicates()
print(f"Passing: {before} -> {len(passing)}")

before = len(defense)
defense = defense.drop_duplicates()
print(f"Defense: {before} -> {len(defense)}")

before = len(possession)
possession = possession.drop_duplicates()
print(f"Possession: {before} -> {len(possession)}")

before = len(shooting)
shooting = shooting.drop_duplicates()
print(f"Shooting: {before} -> {len(shooting)}")

before = len(misc)
misc = misc.drop_duplicates()
print(f"Misc: {before} -> {len(misc)}")

## 3. Merge All Tables

In [ ]:
# Key columns for merging
KEY_COLS = ['player', 'team', 'season', 'league']

# Start with standard stats
df = standard.copy()
print(f"Starting shape: {df.shape}")

# Merge other tables
tables = [
    ('passing', passing),
    ('defense', defense),
    ('possession', possession),
    ('shooting', shooting),
    ('misc', misc)
]

for name, table in tables:
    # Get columns to add (exclude duplicates except keys)
    existing_cols = set(df.columns)
    new_cols = [c for c in table.columns if c not in existing_cols or c in KEY_COLS]
    
    # Merge
    df = df.merge(
        table[new_cols],
        on=KEY_COLS,
        how='left',
        suffixes=('', f'_{name}')
    )
    print(f"After {name}: {df.shape}")

print(f"\nFinal merged shape: {df.shape}")

## 4. Clean Data

In [ ]:
# Rename columns for clarity
df = df.rename(columns={
    'player': 'Player',
    'team': 'Squad',
    'pos': 'Pos',
    'age': 'Age',
    'nation': 'Nation',
    'league': 'League',
    'season': 'Season'
})

print(f"Columns: {list(df.columns[:20])}...")

In [ ]:
# Clean age column (extract numeric age)
def clean_age(age_str):
    if pd.isna(age_str):
        return np.nan
    try:
        # Handle "25-123" format (age-days)
        if isinstance(age_str, str) and '-' in age_str:
            return int(age_str.split('-')[0])
        return int(float(age_str))
    except:
        return np.nan

df['Age'] = df['Age'].apply(clean_age)
print(f"Age range: {df['Age'].min()} - {df['Age'].max()}")

In [ ]:
# Filter: minimum playing time (at least 2 x 90 minutes)
MIN_90S = 2.0

# Find 90s column
nineties_col = None
for col in df.columns:
    if '90s' in str(col).lower():
        nineties_col = col
        break

if nineties_col:
    before = len(df)
    df = df[df[nineties_col] >= MIN_90S]
    print(f"Filtered by {nineties_col} >= {MIN_90S}: {before} -> {len(df)} rows")
else:
    print("Warning: 90s column not found")

In [ ]:
# Remove goalkeepers (they have very different stats)
before = len(df)
df = df[~df['Pos'].str.contains('GK', na=False)]
print(f"Removed goalkeepers: {before} -> {len(df)} rows")

print(f"\nPosition distribution after filtering:")
print(df['Pos'].value_counts())

## 5. Handle Duplicates & Aggregate

In [ ]:
# Check duplicates after cleaning
print("=" * 60)
print("DUPLICATE CHECK - After Cleaning")
print("=" * 60)

# Total rows vs unique players
print(f"\nTotal rows: {len(df)}")
print(f"Unique players: {df['Player'].nunique()}")

# Players appearing multiple times
player_counts = df['Player'].value_counts()
multi_appearance = player_counts[player_counts > 1]
print(f"Players with multiple rows: {len(multi_appearance)}")

# Show examples
if len(multi_appearance) > 0:
    print(f"\nExamples of players with multiple entries:")
    for player in multi_appearance.head(5).index:
        player_rows = df[df['Player'] == player][['Player', 'Squad', 'Season', 'Pos']]
        print(f"\n  {player}:")
        for _, row in player_rows.iterrows():
            print(f"    - {row['Season']} | {row['Squad']} | {row['Pos']}")

In [ ]:
# Identify numeric columns for aggregation
id_cols = ['Player', 'Squad', 'Pos', 'Nation', 'League', 'Age', 'born', 'Season']
numeric_cols = [c for c in df.columns if c not in id_cols 
                and df[c].dtype in ['int64', 'float64']]

print(f"Numeric columns to aggregate: {len(numeric_cols)}")

In [ ]:
# Aggregate: For players with multiple entries, take WEIGHTED AVERAGE by 90s played
# This gives more weight to seasons/stints where they played more

def weighted_aggregate(group):
    """Aggregate player stats weighted by playing time."""
    result = {}
    
    # For ID columns, take the most recent (last) entry
    result['Squad'] = group['Squad'].iloc[-1]
    result['Pos'] = group['Pos'].iloc[-1]
    result['Nation'] = group['Nation'].iloc[-1] if 'Nation' in group.columns else None
    result['League'] = group['League'].iloc[-1]
    result['Age'] = group['Age'].iloc[-1]
    
    # Get weights (90s played)
    weights = group['90s'].values if '90s' in group.columns else np.ones(len(group))
    weights = np.maximum(weights, 0.1)  # Avoid division by zero
    
    # Weighted average for numeric columns
    for col in numeric_cols:
        if col in group.columns:
            values = group[col].values
            # Handle NaN: use only non-NaN values
            mask = ~np.isnan(values)
            if mask.any():
                result[col] = np.average(values[mask], weights=weights[mask])
            else:
                result[col] = np.nan
    
    return pd.Series(result)

print("Aggregating players (weighted by playing time)...")
df_agg = df.groupby('Player').apply(weighted_aggregate).reset_index()

print(f"\nAfter aggregation: {df_agg.shape}")
print(f"Unique players: {df_agg['Player'].nunique()}")

In [ ]:
# FINAL DUPLICATE CHECK
print("=" * 60)
print("FINAL DUPLICATE CHECK")
print("=" * 60)

final_dups = df_agg['Player'].duplicated().sum()
print(f"\nDuplicate players remaining: {final_dups}")

if final_dups > 0:
    print("WARNING: Still have duplicates!")
    print(df_agg[df_agg['Player'].duplicated(keep=False)][['Player', 'Squad', 'Pos']])
else:
    print("✓ All players are unique!")

In [ ]:
# Drop rows with too many missing values
threshold = 0.5  # At least 50% of numeric columns must have values
min_non_null = int(len(numeric_cols) * threshold)

before = len(df_agg)
df_agg = df_agg.dropna(subset=numeric_cols, thresh=min_non_null)
print(f"Dropped rows with >50% missing: {before} -> {len(df_agg)} players")

# Fill remaining NaN with 0
df_agg[numeric_cols] = df_agg[numeric_cols].fillna(0)

print(f"\nFinal dataset: {df_agg.shape}")
print(f"\nPosition breakdown:")
print(df_agg['Pos'].value_counts())

## 6. Create Feature List

In [ ]:
# Identify feature columns
exclude_cols = ['Player', 'Squad', 'Pos', 'Nation', 'League', 'Age', 'born', 'Season']
feature_cols = [c for c in df_agg.columns if c not in exclude_cols 
                and df_agg[c].dtype in ['int64', 'float64']]

print(f"Feature columns: {len(feature_cols)}")

In [ ]:
# Feature name mappings
feature_mapping = {
    'Playing Time': 'Matches Played',
    'Playing Time.1': 'Starts',
    'Playing Time.2': 'Minutes',
    'Playing Time.3': 'Minutes per Match',
    'Performance': 'Goals',
    'Performance.1': 'Assists',
    'Performance.2': 'Goals + Assists',
    'Performance.3': 'Non-Penalty Goals',
    'Performance.4': 'Penalty Goals',
    'Performance.5': 'Penalty Attempts',
    'Performance.6': 'Yellow Cards',
    'Performance.7': 'Red Cards',
    'Expected': 'xG (Expected Goals)',
    'Expected.1': 'npxG (Non-Penalty xG)',
    'Expected.2': 'xAG (Expected Assists)',
    'Expected.3': 'npxG + xAG',
    'Expected.4': 'npxG per Shot',
    'Progression': 'Progressive Carries',
    'Progression.1': 'Progressive Passes',
    'Progression.2': 'Progressive Passes Received',
    'Per 90 Minutes': 'Goals per 90',
    'Per 90 Minutes.1': 'Assists per 90',
    'Per 90 Minutes.2': 'G+A per 90',
    'Per 90 Minutes.3': 'Non-Penalty Goals per 90',
    'Per 90 Minutes.4': 'G+A-PK per 90',
    'Per 90 Minutes.5': 'xG per 90',
    'Per 90 Minutes.6': 'xAG per 90',
    'Per 90 Minutes.7': 'xG+xAG per 90',
    'Per 90 Minutes.8': 'npxG per 90',
    'Per 90 Minutes.9': 'npxG+xAG per 90',
    '90s': '90-minute periods played',
    'Standard': 'Shots Total',
    'Standard.1': 'Shots on Target',
    'Standard.2': 'SoT%',
    'Standard.3': 'Shots per 90',
    'Standard.4': 'SoT per 90',
    'Standard.5': 'Goals per Shot',
    'Standard.6': 'Goals per SoT',
    'Standard.7': 'Avg Shot Distance',
    'Standard.8': 'Free Kick Shots',
    'Standard.9': 'PK Made',
    'Standard.10': 'PK Attempted',
    'Standard.11': 'xG per Shot',
    'Total': 'Passes Completed',
    'Total.1': 'Passes Attempted',
    'Total.2': 'Pass %',
    'Total.3': 'Total Pass Distance',
    'Total.4': 'Progressive Pass Distance',
    'Short': 'Short Passes Completed',
    'Short.1': 'Short Passes Attempted',
    'Short.2': 'Short Pass %',
    'Medium': 'Medium Passes Completed',
    'Medium.1': 'Medium Passes Attempted',
    'Medium.2': 'Medium Pass %',
    'Long': 'Long Passes Completed',
    'Long.1': 'Long Passes Attempted',
    'Long.2': 'Long Pass %',
    'KP': 'Key Passes',
    '1/3': 'Passes into Final Third',
    'PPA': 'Passes into Penalty Area',
    'CrsPA': 'Crosses into Penalty Area',
    'PrgP': 'Progressive Passes',
    'Touches': 'Total Touches',
    'Touches.1': 'Def Penalty Area Touches',
    'Touches.2': 'Def Third Touches',
    'Touches.3': 'Mid Third Touches',
    'Touches.4': 'Att Third Touches',
    'Touches.5': 'Att Penalty Area Touches',
    'Touches.6': 'Live Ball Touches',
    'Take-Ons': 'Dribbles Attempted',
    'Take-Ons.1': 'Successful Dribbles',
    'Take-Ons.2': 'Dribble Success %',
    'Take-Ons.3': 'Times Tackled',
    'Take-Ons.4': 'Tackled %',
    'Carries': 'Total Carries',
    'Carries.1': 'Carry Distance',
    'Carries.2': 'Progressive Carry Distance',
    'Carries.3': 'Progressive Carries',
    'Carries.4': 'Carries into Final Third',
    'Carries.5': 'Carries into Penalty Area',
    'Carries.6': 'Miscontrols',
    'Carries.7': 'Dispossessed',
    'Receiving': 'Passes Received',
    'Receiving.1': 'Progressive Passes Received',
    'Tackles': 'Tackles',
    'Tackles.1': 'Tackles Won',
    'Tackles.2': 'Def Third Tackles',
    'Tackles.3': 'Mid Third Tackles',
    'Tackles.4': 'Att Third Tackles',
    'Challenges': 'Dribblers Tackled',
    'Challenges.1': 'Dribbles Challenged',
    'Challenges.2': 'Challenge %',
    'Challenges.3': 'Dribblers Past',
    'Blocks': 'Total Blocks',
    'Blocks.1': 'Shots Blocked',
    'Blocks.2': 'Passes Blocked',
    'Int': 'Interceptions',
    'Tkl+Int': 'Tackles + Interceptions',
    'Clr': 'Clearances',
    'Err': 'Errors Leading to Shot',
    'Aerial Duels': 'Aerials Won',
    'Aerial Duels.1': 'Aerials Lost',
    'Aerial Duels.2': 'Aerial Win %'
}

# Write feature file
with open(DATA_DIR / 'clustering_features_allpos.txt', 'w', encoding='utf-8') as f:
    f.write('# All Positions Clustering Features\n')
    f.write('# ==================================\n\n')
    for col in feature_cols:
        readable = feature_mapping.get(col, col)
        f.write(f"{col}          # {readable}\n")

print(f"Saved feature list to {DATA_DIR / 'clustering_features_allpos.txt'}")

## 7. Save Processed Data

In [ ]:
# Save the processed data
output_file = DATA_DIR / 'players_allpos_multiseason.csv'
df_agg.to_csv(output_file, index=False)

print(f"\n{'='*60}")
print(f"SAVED: {output_file}")
print(f"{'='*60}")
print(f"Total unique players: {len(df_agg)}")
print(f"Total features: {len(feature_cols)}")
print(f"\nPosition breakdown:")
for pos, count in df_agg['Pos'].value_counts().items():
    print(f"  {pos:10s}: {count:4d} players")

In [ ]:
# Final sanity check
print("\nSample players:")
sample = df_agg.sample(10)[['Player', 'Squad', 'Pos', 'Age']]
display(sample)